In [1]:
# importing all libraries needed 

import pandas as pd
import numpy as np
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import xml
import json
from pandas.io.json import json_normalize
import folium
# collecting website
url = requests.get ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

Creating table using Postal Code, Borough, Neighbourhood (Canada's M postal codes)

In [3]:
table_data = soup.find('table')
fields = table_data.find_all('td')
postcode = []
borough=[]
neighbourhood = []
for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
    
df_ca = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_ca.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_ca.head(10), df_ca.tail(10)

(  Postcode           Borough     Neighbourhood
 0      M1A      Not assigned      Not assigned
 1      M2A      Not assigned      Not assigned
 2      M3A        North York         Parkwoods
 3      M4A        North York  Victoria Village
 4      M5A  Downtown Toronto      Harbourfront
 5      M5A  Downtown Toronto       Regent Park
 6      M6A        North York  Lawrence Heights
 7      M6A        North York    Lawrence Manor
 8      M7A      Queen's Park      Not assigned
 9      M8A      Not assigned      Not assigned,
     Postcode       Borough             Neighbourhood
 278      M4Z  Not assigned              Not assigned
 279      M5Z  Not assigned              Not assigned
 280      M6Z  Not assigned              Not assigned
 281      M7Z  Not assigned              Not assigned
 282      M8Z     Etobicoke  Kingsway Park South West
 283      M8Z     Etobicoke                 Mimico NW
 284      M8Z     Etobicoke        The Queensway West
 285      M8Z     Etobicoke     Royal Y

Removing 'Not assigned' from post codes

In [4]:
df_ca['Borough'].replace('Not assigned', np.nan, inplace = True)
df_ca.dropna(subset=['Borough'], inplace =True)
df_ca.head(10), df_ca.tail(10)

(   Postcode           Borough     Neighbourhood
 2       M3A        North York         Parkwoods
 3       M4A        North York  Victoria Village
 4       M5A  Downtown Toronto      Harbourfront
 5       M5A  Downtown Toronto       Regent Park
 6       M6A        North York  Lawrence Heights
 7       M6A        North York    Lawrence Manor
 8       M7A      Queen's Park      Not assigned
 10      M9A         Etobicoke  Islington Avenue
 11      M1B       Scarborough             Rouge
 12      M1B       Scarborough           Malvern,
     Postcode    Borough             Neighbourhood
 269      M8Y  Etobicoke                 Mimico NE
 270      M8Y  Etobicoke            Old Mill South
 271      M8Y  Etobicoke        The Queensway East
 272      M8Y  Etobicoke     Royal York South East
 273      M8Y  Etobicoke                  Sunnylea
 282      M8Z  Etobicoke  Kingsway Park South West
 283      M8Z  Etobicoke                 Mimico NW
 284      M8Z  Etobicoke        The Queensway West
 

Linking renewed list together

In [ ]:
df_ca_new= df_ca.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_ca_new.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_ca_new

Assigning neighbourhood name to Not assigned neighbourhood from Queen's Park. Assumption Made

In [16]:
df_ca_new['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace =True)
df_ca_new.head(12)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"
